In [11]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# OWRI FRAMEWORK

In [12]:
# results save path
base_result_path = '../results/hauge/LSTM'
exp_name = 'multivariate_AE_weighted_real_time_results.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [13]:
with open('../data/hauge/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [14]:
# load data of correlated results from pickle file
with open('../results/hauge/outlier_scores/AE/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [15]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[trajectory][direction]['raw']:
        intersection = results[trajectory][direction]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [16]:
def preprocess_df(df,top_corr_df, n_obs, n_features, sequence_length):
    #do scaling:
    scaler = StandardScaler()
    train_portion = 0.8
    test_portion = 0.2
    df_train = df[:math.ceil(len(df)*train_portion)].values
    df_test = df[math.ceil(len(df)*(train_portion)):].values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features:]
    test_X, test_y = df_test[:, :n_obs], df_test[:, -n_features:]
    scl = scaler.fit(train_X) # fit only on training data
    train_X = scl.transform(train_X)
    test_X = scl.transform(test_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    test_X = test_X.reshape((test_X.shape[0], sequence_length, n_features))

    # add top correlated features
    corr_array = top_corr_df.values
    corr_tiled_train = np.tile(corr_array, (train_X.shape[0], sequence_length, 1, 1))
    corr_tiled_test = np.tile(corr_array, (test_X.shape[0], sequence_length, 1, 1))
    train_X = np.concatenate([train_X[:, :, :, np.newaxis], corr_tiled_train], axis=3) # add outlier dimension
    train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],-1) # reshape to 3D
    test_X = np.concatenate([test_X[:, :, :, np.newaxis], corr_tiled_test], axis=3) # add outlier dimension
    test_X = test_X.reshape(test_X.shape[0],test_X.shape[1],-1) # reshape to 3D

    return train_X, train_y, test_X, test_y, scaler

In [17]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [18]:
def get_top_corr_features(top_corr_df, top_k_col):
    # create new coorelation df with top k correlated intersections for each intersection
    new_corr_df = []
    for col in top_corr_df.columns:
        new_corr_df.append(top_corr_df[col].sort_values(ascending=False)[:top_k_col].values)
    new_corr_df = pd.DataFrame(new_corr_df, index=top_corr_df.columns)
    return new_corr_df

In [19]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer
        return out

In [20]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=250):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=batch_size, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        ep_start = time.time()
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)

    return history
    

In [21]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=64, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    return preds

In [22]:
# declare variables
thresholds = [0,0.25,0.5,0.75,1]
epoch = 100
batch_size = 32
learning_rate = 0.01
hidden_size = 64
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [23]:
errors={}  # define a dictionary to store the errors
dfs={} # Define a dictionary to store the dataframes
errors = {} # Define a dictionary to store the errors
intersection_arrays = [] # Define a list to store the intersection arrays
for trajectory in results.keys(): # Loop over all trajectories
    print("\nStarting trajectory: {}".format(trajectory))  
    # Define a dictionary to store the errors for this trajectory
    errors[trajectory]={}
    # Loop over all directions
    for direction in results[trajectory]:
        print("\nStarting direction: {}".format(direction))
        # Define a dictionary to store the errors for this trajectory and direction
        errors[trajectory][direction]={}
        # Loop over all thresholds
        for threshold in thresholds:
            print("Starting threshold: {}".format(threshold))
            # Define a dictionary to store the errors for this trajectory, direction, and threshold
            errors[trajectory][direction][threshold]={}
            # ------------------------------------ data processing ---------------------------------------- #
            df = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
            ae_score = correlated_results[trajectory][direction] # AE scores of the current trajectory and direction
            top_k_col = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections to use
            if top_k_col==0: # if threshold is 0, then use the target intersection only
                top_k_col=1
            top_corr_df = ae_score.corr()[df.columns.to_list()] # rearrange the correlation matrix
            top_corr_df = get_top_corr_features(top_corr_df, top_k_col) # get the top k correlated intersections
            n_features = len(df.columns) # number of features (correlated intersections)
            n_obs = sequence_length * n_features # number of columns in the input
            reframed = series_to_supervised(df, sequence_length, output_pred)
            train_X, train_y, test_X, test_y, scl = preprocess_df(reframed,top_corr_df, n_obs, n_features, sequence_length)
            train_X, train_y, test_X, test_y = train_X.astype('float32'), train_y.astype('float32'), test_X.astype('float32'), test_y.astype('float32')

            # # # ------------------------------------ modelling ---------------------------------------------- #
            # define model, loss function and optimizer
            model = LSTM_uni(input_dim = train_X.shape[-1], hidden_dim = hidden_size, layer_dim = num_layers, output_dim = train_y.shape[1], dropout_prob= dropout)
            model = model.to(device)
            loss_fn = torch.nn.MSELoss()
            optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
            start = time.time()
            history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size = batch_size, epochs=epoch)
            end = time.time()
            print("Training time: {}".format(end-start))

            # ------------------------------------ evaluation ---------------------------------------------- #
            yhat = model_evaluation(model, test_X , device)
            errors[trajectory][direction][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
            errors[trajectory][direction][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
            errors[trajectory][direction][threshold]['history'] = history
            errors[trajectory][direction][threshold]['df'] = {"Real":test_y,"Predicted":yhat}
            errors[trajectory][direction][threshold]['train_time'] = end-start


# save errors in save path as pickle file
with open(results_save_path, 'wb') as handle:
    pickle.dump(errors, handle)


Starting trajectory: T1

Starting direction: North
Starting threshold: 0


In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[trajectory][direction].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[trajectory][direction][threshold]['RMSE']

In [ ]:
AE_results

{'T1_North_0.5': 13.207560391278088,
 'T1_South_0.5': 15.087792905225621,
 'T2_South_0.5': 13.155181768909424,
 'T2_North_0.5': 15.316885931184128}

# EXTRA

In [292]:
c = [a.cpu().detach().numpy(),b.cpu().detach().numpy()]

In [293]:
d = np.vstack(c)

In [300]:
a.shape

torch.Size([64, 9])

In [304]:
temp = test_y[:64,:].shape

In [251]:
test_y

torch.Size([64, 9])

In [264]:
mean_squared_error(bt.cpu().detach().numpy(),y.cpu().detach().numpy())

35.592785

In [266]:
loss_fn(bt.reshape(-1),y.reshape(-1))

tensor(35.5928, device='mps:0', grad_fn=<MseLossBackward0>)

torch.Size([576])

In [239]:
train_X.shape

(107021, 12, 90)

In [241]:
model

LSTM_uni(
  (lstm): LSTM(9, 64, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=9, bias=True)
)

In [91]:
b = ae_score.corr().values
b.shape

(15, 15)

In [144]:
b_tiled = np.tile(b, (12, 1, 1))

In [151]:
b_tiled.shape

(12, 15, 15)

In [145]:
b_tiled_temp = np.tile(b, (78327, 12, 1, 1))

In [146]:
b_tiled_temp.shape

(78327, 12, 15, 15)

In [147]:
b_tiled_temp

array([[[[1.        , 0.52009281, 0.59531289, ..., 0.47456496,
          0.50346994, 0.55122234],
         [0.52009281, 1.        , 0.51728952, ..., 0.44649454,
          0.54981047, 0.59161937],
         [0.59531289, 0.51728952, 1.        , ..., 0.46227573,
          0.48162972, 0.52886381],
         ...,
         [0.47456496, 0.44649454, 0.46227573, ..., 1.        ,
          0.44934213, 0.51765878],
         [0.50346994, 0.54981047, 0.48162972, ..., 0.44934213,
          1.        , 0.58109163],
         [0.55122234, 0.59161937, 0.52886381, ..., 0.51765878,
          0.58109163, 1.        ]],

        [[1.        , 0.52009281, 0.59531289, ..., 0.47456496,
          0.50346994, 0.55122234],
         [0.52009281, 1.        , 0.51728952, ..., 0.44649454,
          0.54981047, 0.59161937],
         [0.59531289, 0.51728952, 1.        , ..., 0.46227573,
          0.48162972, 0.52886381],
         ...,
         [0.47456496, 0.44649454, 0.46227573, ..., 1.        ,
          0.44934213, 0.5

In [133]:
b_tiled.shape

(12, 15, 15)

In [153]:
c = np.concatenate([train_X[:, :, :, np.newaxis], b_tiled_temp], axis=3)

In [152]:
train_X.shape

(78327, 12, 15)

In [149]:
a.shape

(12, 15)

In [139]:
c[1]

array([[24.        ,  1.        ,  0.52009281,  0.59531289,  0.4905096 ,
         0.59030765,  0.53083411,  0.42160134,  0.44977888,  0.41419196,
         0.43560344,  0.54621173,  0.5127304 ,  0.47456496,  0.50346994,
         0.55122234],
       [42.        ,  0.52009281,  1.        ,  0.51728952,  0.45122571,
         0.49783694,  0.3569048 ,  0.40421719,  0.3800037 ,  0.30118805,
         0.41944514,  0.36872095,  0.51981267,  0.44649454,  0.54981047,
         0.59161937],
       [55.        ,  0.59531289,  0.51728952,  1.        ,  0.48578304,
         0.5503211 ,  0.56126153,  0.41188402,  0.47546262,  0.42028723,
         0.40471204,  0.55828677,  0.49244704,  0.46227573,  0.48162972,
         0.52886381],
       [26.        ,  0.4905096 ,  0.45122571,  0.48578304,  1.        ,
         0.49171092,  0.41857639,  0.38567624,  0.39845473,  0.33828968,
         0.37756179,  0.43709072,  0.42578968,  0.40196755,  0.42904218,
         0.48198089],
       [14.        ,  0.59030765,  0

In [140]:
train_X[0]

array([[28., 24., 54., 44., 26., 34., 31., 30., 56., 59., 36.,  6., 30.,
         9., 15.],
       [24., 42., 55., 26., 14., 38., 29., 31., 57., 29., 21.,  4., 21.,
         6.,  6.],
       [40., 41., 52., 30., 17., 27., 25., 22., 48., 33., 30.,  6., 20.,
         3.,  7.],
       [46., 43., 53., 41., 18., 42., 20., 13., 51., 55., 22.,  0., 19.,
         5.,  8.],
       [ 6., 46., 60., 32.,  5., 27., 34., 34.,  9., 57., 21.,  0., 27.,
        12., 19.],
       [41., 63., 84., 38., 22., 28., 18., 22., 48., 55., 26.,  3., 22.,
         0., 14.],
       [34., 51., 69., 30., 10., 27., 25., 11., 42., 54., 30.,  7., 22.,
        10., 13.],
       [27., 39., 45., 33.,  3., 38., 26., 36., 34., 36., 34.,  0., 20.,
         5.,  5.],
       [28., 63., 74., 27., 11., 37., 37., 26., 44., 54., 35.,  7., 10.,
         6., 19.],
       [35., 51., 52., 34., 19., 18., 23., 38., 47., 40., 24.,  5., 23.,
        13., 15.],
       [31., 40., 36., 30., 21., 31., 36., 29., 50., 52., 31.,  3., 22.,
       

In [128]:
c = np.concatenate([a[:, :, np.newaxis], b_reshaped], axis=2)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 12 and the array at index 1 has size 1

In [104]:
# stack each row of b with each element of a
a[0].shape

(15,)

In [119]:
np.hstack((a[0].reshape(-1,1),b))

array([[28.        ,  1.        ,  0.52009281,  0.59531289,  0.4905096 ,
         0.59030765,  0.53083411,  0.42160134,  0.44977888,  0.41419196,
         0.43560344,  0.54621173,  0.5127304 ,  0.47456496,  0.50346994,
         0.55122234],
       [24.        ,  0.52009281,  1.        ,  0.51728952,  0.45122571,
         0.49783694,  0.3569048 ,  0.40421719,  0.3800037 ,  0.30118805,
         0.41944514,  0.36872095,  0.51981267,  0.44649454,  0.54981047,
         0.59161937],
       [54.        ,  0.59531289,  0.51728952,  1.        ,  0.48578304,
         0.5503211 ,  0.56126153,  0.41188402,  0.47546262,  0.42028723,
         0.40471204,  0.55828677,  0.49244704,  0.46227573,  0.48162972,
         0.52886381],
       [44.        ,  0.4905096 ,  0.45122571,  0.48578304,  1.        ,
         0.49171092,  0.41857639,  0.38567624,  0.39845473,  0.33828968,
         0.37756179,  0.43709072,  0.42578968,  0.40196755,  0.42904218,
         0.48198089],
       [26.        ,  0.59030765,  0

In [126]:
b

array([[1.        , 0.52009281, 0.59531289, 0.4905096 , 0.59030765,
        0.53083411, 0.42160134, 0.44977888, 0.41419196, 0.43560344,
        0.54621173, 0.5127304 , 0.47456496, 0.50346994, 0.55122234],
       [0.52009281, 1.        , 0.51728952, 0.45122571, 0.49783694,
        0.3569048 , 0.40421719, 0.3800037 , 0.30118805, 0.41944514,
        0.36872095, 0.51981267, 0.44649454, 0.54981047, 0.59161937],
       [0.59531289, 0.51728952, 1.        , 0.48578304, 0.5503211 ,
        0.56126153, 0.41188402, 0.47546262, 0.42028723, 0.40471204,
        0.55828677, 0.49244704, 0.46227573, 0.48162972, 0.52886381],
       [0.4905096 , 0.45122571, 0.48578304, 1.        , 0.49171092,
        0.41857639, 0.38567624, 0.39845473, 0.33828968, 0.37756179,
        0.43709072, 0.42578968, 0.40196755, 0.42904218, 0.48198089],
       [0.59030765, 0.49783694, 0.5503211 , 0.49171092, 1.        ,
        0.5057194 , 0.41738819, 0.45516972, 0.40246901, 0.42746746,
        0.49505185, 0.50251473, 0.47448538, 

In [88]:
# merge a and b to make it 12 x 15 x 15 array
c = np.concatenate((a,b), axis=0)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 15

In [89]:
c

array([[28.        , 24.        , 54.        , 44.        , 26.        ,
        34.        , 31.        , 30.        , 56.        , 59.        ,
        36.        ,  6.        , 30.        ,  9.        , 15.        ],
       [24.        , 42.        , 55.        , 26.        , 14.        ,
        38.        , 29.        , 31.        , 57.        , 29.        ,
        21.        ,  4.        , 21.        ,  6.        ,  6.        ],
       [40.        , 41.        , 52.        , 30.        , 17.        ,
        27.        , 25.        , 22.        , 48.        , 33.        ,
        30.        ,  6.        , 20.        ,  3.        ,  7.        ],
       [46.        , 43.        , 53.        , 41.        , 18.        ,
        42.        , 20.        , 13.        , 51.        , 55.        ,
        22.        ,  0.        , 19.        ,  5.        ,  8.        ],
       [ 6.        , 46.        , 60.        , 32.        ,  5.        ,
        27.        , 34.        , 34.        , 

In [67]:
ae_score.corr().columns.to_list()

['K704',
 'K702',
 'K703',
 'K159',
 'K182',
 'K183',
 'K128',
 'K139',
 'K104',
 'K101',
 'K206',
 'K074',
 'K414',
 'K415',
 'K250']

In [29]:
ae_score.corr()[isct_inc]

,K703,K704,K183,K206,K182,K250,K702,K074
K704,0.595313,1.000000,0.530834,0.546212,0.590308,0.551222,0.520093,0.512730
K702,0.517290,0.520093,0.356905,0.368721,0.497837,0.591619,1.000000,0.519813
K703,1.000000,0.595313,0.561262,0.558287,0.550321,0.528864,0.517290,0.492447
K159,0.485783,0.490510,0.418576,0.437091,0.491711,0.481981,0.451226,0.425790
K182,0.550321,0.590308,0.505719,0.495052,1.000000,0.548973,0.497837,0.502515
K183,0.561262,0.530834,1.000000,0.556446,0.505719,0.452424,0.356905,0.460386
K128,0.411884,0.421601,0.415147,0.392648,0.417388,0.433686,0.404217,0.425242
K139,0.475463,0.449779,0.402664,0.423490,0.455170,0.424486,0.380004,0.397950
K104,0.420287,0.414192,0.424533,0.414931,0.402469,0.368167,0.301188,0.361701
K101,0.404712,0.435603,0.365576,0.425814,0.427467,0.446986,0.419445,0.461534


In [59]:
data.columns.to_list()

['K704',
 'K702',
 'K703',
 'K159',
 'K182',
 'K183',
 'K128',
 'K139',
 'K104',
 'K101',
 'K206',
 'K074',
 'K414',
 'K415',
 'K250']

In [58]:
data

,K704,K702,K703,K159,K182,K183,K128,K139,K104,K101,K206,K074,K414,K415,K250
timestamp,,,,,,,,,,,,,,,
2018-01-01 01:00:00,28.0,24.0,54.0,44.0,26.0,34.0,31.0,30.0,56.0,59.0,36.0,6.0,30.0,9.0,15.0
2018-01-01 01:05:00,24.0,42.0,55.0,26.0,14.0,38.0,29.0,31.0,57.0,29.0,21.0,4.0,21.0,6.0,6.0
2018-01-01 01:10:00,40.0,41.0,52.0,30.0,17.0,27.0,25.0,22.0,48.0,33.0,30.0,6.0,20.0,3.0,7.0
2018-01-01 01:15:00,46.0,43.0,53.0,41.0,18.0,42.0,20.0,13.0,51.0,55.0,22.0,0.0,19.0,5.0,8.0
2018-01-01 01:20:00,6.0,46.0,60.0,32.0,5.0,27.0,34.0,34.0,9.0,57.0,21.0,0.0,27.0,12.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 23:35:00,4.0,4.0,9.0,10.0,13.0,4.0,10.0,19.0,14.0,21.0,13.0,3.0,4.0,1.0,2.0
2020-03-31 23:40:00,1.0,5.0,5.0,6.0,5.0,7.0,12.0,5.0,14.0,13.0,12.0,3.0,7.0,3.0,3.0
2020-03-31 23:45:00,8.0,7.0,4.0,7.0,4.0,12.0,6.0,8.0,17.0,12.0,13.0,3.0,19.0,5.0,3.0


In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[trajectory][direction].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[trajectory][direction][threshold]['RMSE']

In [ ]:
AE_results

{'T1_North_0': 10.59151633354107,
 'T1_North_0.25': 10.196899412467983,
 'T1_North_0.5': 9.955075986889941,
 'T1_North_0.75': 10.05299665551894,
 'T1_North_1': 9.830371060706703,
 'T1_South_0': 18.641919656471167,
 'T1_South_0.25': 17.971930531215893,
 'T1_South_0.5': 17.78031504964693,
 'T1_South_0.75': 17.8761927808514,
 'T1_South_1': 17.471410650736853,
 'T2_South_0': 15.783835842420112,
 'T2_South_0.25': 14.955686164952139,
 'T2_South_0.5': 14.87429694295857,
 'T2_South_0.75': 14.916430099779108,
 'T2_South_1': 14.930474792897037,
 'T2_North_0': 15.473913778461938,
 'T2_North_0.25': 14.940598792684167,
 'T2_North_0.5': 15.136461083171628,
 'T2_North_0.75': 15.01907330839573,
 'T2_North_1': 15.349926838139215}